In [1]:
from pycocotools.coco import COCO

In [2]:
%pylab --no-import-all
%reload_ext autoreload
%autoreload 2
#%matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
import json
from pathlib import Path
from pprint import pprint

In [4]:
import pandas as pd
import numpy as np

In [5]:
json_dir = Path('./data/training/GT')

In [6]:
data = []

In [7]:
for file in json_dir.iterdir():
    if str(file).endswith('json'):
        fdata = json.load(file.open())
        data.extend(fdata)

In [8]:
data[0]

{u'annotations': [{u'class': u'Person',
   u'height': 130.20504878900792,
   u'width': 143.9422787071143,
   u'x': 142.1504661091004,
   u'y': 168.43038421330382},
  {u'class': u'Head', u'x': 185.75123932743793, u'y': 213.82297002965524},
  {u'class': u'Left Shoulder',
   u'x': 219.19840782369684,
   u'y': 203.6693653075766},
  {u'class': u'Left Elbow', u'x': 246.6728676599095, u'y': 222.78203301972457},
  {u'class': u'Left Hand', u'x': 251.4510345879465, u'y': 256.82647238198814},
  {u'class': u'Right Shoulder',
   u'x': 167.83311334729922,
   u'y': 240.70015899986328},
  {u'class': u'Right Elbow',
   u'x': 170.81946767732234,
   u'y': 273.5500566301176},
  {u'class': u'Right Hand',
   u'x': 221.58749128771532,
   u'y': 268.7718897020806}],
 u'class': u'image',
 u'filename': u'data/training/10_17_T3_K1/frame00020.jpg'}

In [9]:
pdata = pd.DataFrame(data)

In [10]:
pdata[:3]

,annotations,class,filename
0,"[{u'y': 168.430384213, u'width': 143.942278707...",image,data/training/10_17_T3_K1/frame00020.jpg
1,"[{u'y': 166.638571615, u'width': 167.235842481...",image,data/training/10_17_T3_K1/frame00175.jpg
2,"[{u'y': 163.652217285, u'width': 155.887696027...",image,data/training/10_17_T3_K1/frame00200.jpg


### Create coco-style data

In [11]:
coco_data = {"images":[], "annotations":[], "categories":[]}

In [12]:
coco_data["categories"] = [{'supercategory': 'person',
  'id': 1,
  'name': 'person',
  'keypoints': ['nose',
   'left_eye',
   'right_eye',
   'left_ear',
   'right_ear',
   'left_shoulder',
   'right_shoulder',
   'left_elbow',
   'right_elbow',
   'left_wrist',
   'right_wrist',
   'left_hip',
   'right_hip',
   'left_knee',
   'right_knee',
   'left_ankle',
   'right_ankle'],
  'skeleton': [[16, 14],
   [14, 12],
   [17, 15],
   [15, 13],
   [12, 13],
   [6, 12],
   [7, 13],
   [6, 7],
   [6, 8],
   [7, 9],
   [8, 10],
   [9, 11],
   [2, 3],
   [1, 2],
   [1, 3],
   [2, 4],
   [3, 5],
   [4, 6],
   [5, 7]]}]

#### Our Annotation format

In our annotations, we only have head(nose)(0), left shoulder(5), right shoulder(6), left elbow(7), right elbow(8), left wrist(9), right wrist(10), bb, arms crossed, both arms on table,
both arms on body, one arm on body, one arm on table, leaning forward, leaning backward, upright

In [13]:
joint_ann_dict = {"Head":0, "Left Shoulder":5, "Right Shoulder":6, "Left Elbow":7, "Right Elbow":8, "Left Hand":9, "Right Hand": 10}

In [14]:
#h, w = 424, 512

In [18]:
import cv2, os

In [19]:
i = 0
i_ann = 0

for i, item in pdata.iterrows():
    #item = pdata.iloc[i]
    #pprint(item)
    fname = item['filename']
    fname = fname.replace("Kinect","depth")
    fname = fname.replace("ir","ud")
    h, w, _ = cv2.imread(fname).shape
    im = {"height":h, "width":w, "file_name":fname, "id":i}

    coco_data["images"].append(im)

    annpd = pd.DataFrame(item['annotations'])
    
    eps = 15
    for _, person in annpd[annpd["class"]=="Person"].iterrows():
        #print(person)
        ann = {"segmentation":[], "keypoints":[], "iscrowd":0, "num_keypoints":17, "area":0, "id":None, "image_id":i, "bbox":[], "category_id":1}
        ann["bbox"] = [person["x"], person["y"], person["width"], person["height"]]
        ann["area"] = person["width"] * person["height"]
        condition = (annpd["x"] > person["x"]-eps) & (annpd["x"] < person["x"]+person["width"]+eps) & \
                    (annpd["y"] > person["y"]-eps) & (annpd["y"] < person["y"]+person["height"]+eps)
        
        joints = annpd[condition & (annpd["class"]!="Person")]

        kp = np.zeros(51, dtype=np.int)
        for _, row in joints.iterrows():
            if row["class"] in joint_ann_dict:
                idx = 3 * joint_ann_dict[row["class"]]
                kp[idx:idx+3] = int(row["x"]), int(row["y"]), 2
        ann["keypoints"] = kp.tolist()
        ann["id"] = i_ann
        ann["image_id"] = i

        coco_data["annotations"].append(ann)

        i_ann += 1
        #print(kp)

        #print(joints)

In [20]:
fname

u'data/training/10_16_T1_K1/frame13479.jpg'

In [21]:
coco_data['annotations'][:2]

[{'area': 18742.011421860792,
  'bbox': [142.1504661091004,
   168.43038421330382,
   143.9422787071143,
   130.20504878900792],
  'category_id': 1,
  'id': 0,
  'image_id': 0,
  'iscrowd': 0,
  'keypoints': [185,
   213,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   219,
   203,
   2,
   167,
   240,
   2,
   246,
   222,
   2,
   170,
   273,
   2,
   251,
   256,
   2,
   221,
   268,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'num_keypoints': 17,
  'segmentation': []},
 {'area': 21774.951029547803,
  'bbox': [126.6214235929802,
   166.63857161528998,
   167.23584248129458,
   130.20504878900792],
  'category_id': 1,
  'id': 1,
  'image_id': 1,
  'iscrowd': 0,
  'keypoints': [184,
   220,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   225,
   208,
   2,
   166,
   240,
   2,
   246,
   218,
   2,
   150,
   273,
   2,
   250,
   256,
   2,
  

### Write annotations

In [22]:
annfile = Path('annotations/keypoints.json')

In [23]:
json.dump(coco_data, open(str(annfile), mode='w'))

### Test if annotation is correct

In [24]:
from pycocotools.coco import COCO
import numpy as np
#import skimage.io as io
import matplotlib.pyplot as plt

In [25]:
import cv2

In [26]:
annfile = Path('annotations/keypoints.json')

In [27]:
coco = COCO(str(annfile))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [28]:
imgIds = coco.getImgIds(catIds=[1])
imgs = coco.loadImgs(ids = imgIds)

In [29]:
dim = [2, 2]
for i,img in enumerate(imgs):
    fname = img['file_name']
    #fname = fname.replace("Kinect","depth")
    #fname = fname.replace("ir","ud")
    print(fname)
    I = cv2.imread(fname)
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)
    for ann in anns:
        kp = ann['keypoints']
        for j in range(17):
            if kp[3*j+2]==2:
                x, y = kp[3*j], kp[3*j+1]
                cv2.circle(I,(x, y), 5, (0, 255, 0), 3)
        bb = [int(f) for f in ann['bbox']]
        cv2.rectangle(I, (bb[0], bb[1]), (bb[0]+bb[2], bb[1]+bb[3]), (255, 0, 0), 3)
    ff = Path(fname).name
    cv2.imwrite('checkim/{}.jpg'.format(ff), I)

data/training/10_17_T3_K1/frame00020.jpg
data/training/10_17_T3_K1/frame00175.jpg
data/training/10_17_T3_K1/frame00200.jpg
data/training/10_17_T3_K1/frame00334.jpg
data/training/10_17_T3_K1/frame00484.jpg
data/training/10_17_T3_K1/frame00531.jpg
data/training/10_17_T3_K1/frame00553.jpg
data/training/10_17_T3_K1/frame00570.jpg
data/training/10_17_T3_K1/frame00637.jpg
data/training/10_17_T3_K1/frame00673.jpg
data/training/10_17_T3_K1/frame00731.jpg
data/training/10_17_T3_K1/frame00858.jpg
data/training/10_17_T3_K1/frame00883.jpg
data/training/10_17_T3_K1/frame00921.jpg
data/training/10_17_T3_K1/frame00978.jpg
data/training/10_17_T3_K1/frame01003.jpg
data/training/10_17_T3_K1/frame01016.jpg
data/training/10_17_T3_K1/frame01167.jpg
data/training/10_17_T3_K1/frame01331.jpg
data/training/10_17_T3_K1/frame01353.jpg
data/training/10_17_T3_K1/frame01410.jpg
data/training/10_17_T3_K1/frame01416.jpg
data/training/10_17_T3_K1/frame01422.jpg
data/training/10_17_T3_K1/frame01467.jpg
data/training/10